In [1]:
%env CUDA_LAUNCH_BLOCKING=1
%load_ext autoreload
%autoreload complete
# %aimport ...

import sys
import importlib
import warnings

importlib.invalidate_caches()

for mod_name in list(sys.modules):
    if not (str.startswith(mod_name, "robotodo") or str.startswith(mod_name, "tensorspecs")):
        continue
    try:
        importlib.reload(sys.modules[mod_name])
    except Exception as error:
        warnings.warn(f"{error}")


env: CUDA_LAUNCH_BLOCKING=1


In [2]:
from robotodo.engines.isaac._kernel import Kernel

kernel = Kernel([
    "--/log/file=/tmp/todo-isaacsim.log",
    # "--/log/enableStandardStreamOutput=true",
    "--/app/enableStdoutOutput=true",
    # "--no-window",
    # TODO
    # "--/app/content/emptyStageOnStart=false",
    # "--/app/asyncRendering=true",
    "--enable", "isaacsim.exp.full",
], kit_path=__import__("os").path.abspath("./todo-exp.kit"))
# kit_path="isaacsim.exp.base.kit"
kernel.start_app_loop_soon()

[Warning] [simulation_app] Interactive python shell detected but ISAAC_JUPYTER_KERNEL was not set. Problems with asyncio may occur
[Warning] [simulation_app] Please use Isaac Sim Python 3 kernel instead of the default Python 3 Kernel
Loading user config located at: '/home/sysadmin/lab/robotodo/.conda/lib/python3.11/site-packages/omni/data/Kit/Isaac-Sim Base/5.1/user.config.json'
[0.030s] Failed to solve some dependencies locally, syncing with extension registry...
[0.032s] [ext: omni.kit.async_engine-0.0.3] startup
[0.036s] [ext: omni.client.lib-1.1.0] startup
[0.049s] [ext: omni.kit.registry.nucleus-0.0.0] startup
[3.685s] Pulling extension: `omni.usd.schema.scene.visualization-2.0.2` from the registry.
[omni.kit.registry.nucleus] [tid: 140671903851776] creating a link: /home/sysadmin/lab/robotodo/.conda/lib/python3.11/site-packages/omni/data/Kit/Isaac-Sim Base/5.1/exts/3/omni.usd.schema.scene.visualization-2.0.2+107.0.3 -> /home/sysadmin/.local/share/ov/data/exts/v2/omni.usd.schema.s

2025-10-13T08:19:27Z [192,022ms] [Error] [isaacsim.ros2.bridge.impl.extension] ROS2 Bridge startup failed
2025-10-13T08:19:28Z [193,240ms] [Error] [carb.graphics-vulkan.plugin] Could not get NGX parameters block because NGX isn't enabled.
2025-10-13T08:19:28Z [193,240ms] [Error] [carb.graphics-vulkan.plugin] Failed to create NGX context.
2025-10-13T08:19:28Z [193,426ms] [Error] [rtx.optixdenoising73.plugin] Failed to create an Optix adaptor for device 0


[193.267s] app ready


In [3]:
# TODO 
from robotodo.engines.isaac.builder import load_usd_scene

# TODO set up axis
scene = await load_usd_scene(
    "https://omniverse-content-production.s3-us-west-2.amazonaws.com/Assets/Isaac/5.0/Isaac/Environments/Grid/default_environment.usd",
    # "https://omniverse-content-production.s3-us-west-2.amazonaws.com/Assets/Isaac/5.0/Isaac/Environments/Grid/gridroom_black.usd",
    _kernel=kernel,
)

In [4]:
from robotodo.engines.isaac.scene import Scene
from robotodo.engines.isaac.builder import load_urdf, load_usd, build_camera
from robotodo.utils.pose import Pose


# scene = Scene(_kernel=kernel)


panda = await load_urdf(
    "./todo-curobo-sample-configs/franka/franka_description/franka_panda.urdf", 
    scene=scene, 
    fix_root_link=True,
)


mug = await load_usd(
    "https://omniverse-content-production.s3-us-west-2.amazonaws.com/Assets/Isaac/5.0/Isaac/Props/Mugs/SM_Mug_A2.usd",
    scene=scene,
)

mug.collidable = True
mug.rigid_body.enabled = True
mug.pose = Pose(p=[.5, .5, .5])



/home/sysadmin/lab/robotodo/packages/robotodo/engines/isaac/builder.py:250: UserWarning: Failed to acquire context from the USD stage. Stage: Usd.Stage.Open(rootLayer=Sdf.Find('anon:0x37157210:tmp.usda'), sessionLayer=Sdf.Find('anon:0x36224be0:tmp-session.usda'), pathResolverContext=<invalid repr>)
  warnings.warn(
2025-10-13T08:19:30Z [194,896ms] [Error] [omni.usd] Runtime Error: in Create at line 46 of /builds/omniverse/usd-ci/USD/pxr/usd/ar/filesystemWritableAsset.cpp -- Could not create directory '/configuration/' for asset '/configuration/anon:0x37157210:tmp_sensor.usd'

2025-10-13T08:19:30Z [194,896ms] [Error] [omni.usd] Runtime Error: in StartPacking at line 2462 of /builds/omniverse/usd-ci/USD/pxr/usd/usd/crateFile.cpp -- Unable to open /configuration/anon:0x37157210:tmp_sensor.usd for write

2025-10-13T08:19:30Z [194,897ms] [Error] [omni.usd] Runtime Error: in Create at line 46 of /builds/omniverse/usd-ci/USD/pxr/usd/ar/filesystemWritableAsset.cpp -- Could not create directory

[197.132s] Isaac Sim Full App is loaded.


In [5]:
cam = await build_camera(scene=scene)
# cam.pose = Pose.from_lookat([1, 1, 1], p_target=[0, 0, 0])
# cam.pose = Pose.from_lookat([1, 1, 1], p_target=panda.link_poses[..., -1].p)
cam.pose = Pose.from_lookat([1, 1, 1], p_target=mug.pose.p)

In [11]:

import matplotlib.pyplot as plt

kernel.step_app_loop()
cam.pose = Pose.from_lookat([2, 2, 2], p_target=mug.pose.p)
plt.imshow(cam.read_rgba()[0].cpu())

[77.245s] [ext: carb.windowing.plugins-1.0.0] startup
[77.252s] [ext: omni.kit.renderer.init-0.0.0] startup

|---------------------------------------------------------------------------------------------|
| Driver Version: 535.247.01    | Graphics API: Vulkan
|=============================================================================================|
| GPU | Name                             | Active | LDA | GPU Memory | Vendor-ID | LUID       |
|     |                                  |        |     |            | Device-ID | UUID       |
|     |                                  |        |     |            | Bus-ID    |            |
|---------------------------------------------------------------------------------------------|
| 0   | NVIDIA GeForce RTX 3060          | Yes: 0 |     | 12288   MB | 10de      | 0          |
|     |                                  |        |     |            | 2504      | 3157457f.. |
|     |                                  |        |     |           

2025-10-13T05:38:19Z [78,834ms] [Error] [carb.graphics-vulkan.plugin] Could not get NGX parameters block because NGX isn't enabled.
2025-10-13T05:38:19Z [78,834ms] [Error] [carb.graphics-vulkan.plugin] Failed to create NGX context.
2025-10-13T05:38:19Z [78,921ms] [Error] [rtx.optixdenoising73.plugin] Failed to create an Optix adaptor for device 0


[78.937s] [ext: omni.graph.action_nodes_core-1.2.0] startup
[78.943s] [ext: omni.graph.action_nodes-1.51.0] startup
[78.957s] [ext: omni.graph.nodes_core-1.1.0] startup
[78.979s] [ext: omni.graph.action-1.130.0] startup
[78.984s] [ext: omni.graph.nodes-1.171.0] startup
[79.034s] [ext: omni.graph.ui_nodes-1.50.8] startup
[79.128s] [ext: omni.physics-107.3.18] startup
[79.188s] [ext: omni.warp.core-1.8.1] startup
[79.196s] [ext: omni.videoencoding-0.1.2] startup
[79.204s] [ext: omni.physics.physx-107.3.18] startup
[79.210s] [ext: omni.warp-1.8.1] startup
[79.240s] [ext: omni.physics.stageupdate-107.3.18] startup
[79.291s] [ext: omni.graph.scriptnode-1.50.0] startup
[79.303s] [ext: omni.replicator.core-1.12.23] startup
Warp 1.8.1 initialized:
   CUDA Toolkit 12.8, Driver 12.2
   Devices:
     "cpu"      : "CPU"
     "cuda:0"   : "NVIDIA GeForce RTX 3060" (12 GiB, sm_86, mempool enabled)
   Kernel cache:
     /home/sysadmin/.cache/warp/1.8.1


2025-10-13T05:38:20Z [79,470ms] [Error] [omni.graph.core.plugin] Unable to create prim for graph at /Render/PostProcess/SDGPipeline


Warp DeprecationWarning: The `warp.sim` module is deprecated and will be removed in v1.10. Please transition to using the forthcoming Newton library instead.


ValueError: Invalid object in Py_Graph in getWrappedGraphFromNode

In [ ]:
kernel.start_app_loop_soon()

Traceback (most recent call last):
  File "/home/sysadmin/lab/robotodo/.conda/lib/python3.11/site-packages/omni/data/Kit/Isaac-Sim Base/5.1/exts/3/omni.syntheticdata-0.6.13+8131b85d.lx64.r.cp311/omni/syntheticdata/scripts/SyntheticData.py", line 1502, in _post_process_graph_tick
    graph = nodeGraph.get_wrapped_graph()
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
ValueError: Invalid object in Py_Graph in getWrappedGraphFromNode
Scene delegate BAD BUCKET populating mesh2 
	 1 0  0  0
Scene delegate BAD BUCKET populating mesh2 
	 1 0  0  0
Traceback (most recent call last):
  File "/home/sysadmin/lab/robotodo/.conda/lib/python3.11/site-packages/omni/data/Kit/Isaac-Sim Base/5.1/exts/3/omni.syntheticdata-0.6.13+8131b85d.lx64.r.cp311/omni/syntheticdata/scripts/SyntheticData.py", line 1502, in _post_process_graph_tick
    graph = nodeGraph.get_wrapped_graph()
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
ValueError: Invalid object in Py_Graph in getWrappedGraphFromNode
Traceback (most recent call 

In [17]:
import numpy



import matplotlib.pyplot as plt

cam.pose = Pose.from_lookat(
    numpy.random.randint(1, 10, size=3),
    p_target=mug.pose.p,
)
# kernel.step_app_loop()
plt.imshow(cam.read_rgba()[0].cpu())

ValueError: Invalid object in Py_Graph in getWrappedGraphFromNode

ValueError: Invalid object in Py_Graph in getWrappedGraphFromNode

In [179]:
import omni
omni.timeline.acquire_timeline_interface().is_playing()

False

In [169]:
%timeit -n 10 kernel.step_app_loop()


78.4 ms ± 1.8 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [104]:
# # manually flush the fabric data to update Hydra textures
# # self.forward()
# # render the simulation
# # note: we don't call super().render() anymore because they do above operation inside
# #  and we don't want to do it twice. We may remove it once we drop support for Isaac Sim 2022.2.
kernel.get_settings().set("/app/player/playSimulations", False)
kernel._app_framework.update()
# kernel.get_settings().set("/app/player/playSimulations", True)

# self.set_setting("/app/player/playSimulations", False)
# self._app.update()
# self.set_setting("/app/player/playSimulations", True)

In [74]:

s = kernel.get_settings()

%timeit -n 100 s.set("/app/player/playSimulations", False)

817 ns ± 87.9 ns per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [65]:
import omni

timeline = omni.timeline.acquire_timeline_interface()
timeline.is_playing()

False

In [66]:
kernel.hacky_ensure_render()

In [54]:
await scene.step()

In [58]:
kernel.step_app_loop()

In [7]:
import omni

timeline = omni.timeline.acquire_timeline_interface()
timeline.set_auto_update(False)

In [10]:
timeline.stop()
timeline.is_stopped()

False

In [48]:
timeline.pause()

In [8]:
timeline.play()
timeline.is_playing(), timeline.is_auto_updating()

(False, False)

2025-10-11T11:04:52Z [34,433ms] [Error] [omni.physx.plugin] PhysicsUSD: Parse collision - triangle mesh collision (approximation None/MeshSimplification) cannot be a part of a dynamic body, falling back to convexHull approximation: /_02.For dynamic collision please use approximations : convex hull, convex decomposition, box, sphere or SDF approximation.


In [63]:
timeline.is_auto_updating()

True

In [30]:
timeline.forward_one_frame?

Docstring:
forward_one_frame(self: omni.timeline._timeline.ITimeline) -> None


Forwards the timeline by one frame.
Type:      method

In [67]:
timeline.play()
timeline.is_playing(), timeline.is_auto_updating()


(False, True)

In [42]:
# timeline.pause()
timeline.stop()

In [17]:
timeline.forward_one_frame()

2025-10-11T09:50:48Z [531,246ms] [Error] [omni.physx.plugin] PhysicsUSD: Parse collision - triangle mesh collision (approximation None/MeshSimplification) cannot be a part of a dynamic body, falling back to convexHull approximation: /_02.For dynamic collision please use approximations : convex hull, convex decomposition, box, sphere or SDF approximation.


In [38]:
timeline.is_auto_updating()

True

In [27]:
# timeline.set_auto_update(False)
timeline.set_auto_update(True)

In [29]:
timeline.forward_one_frame()

In [69]:
await scene.step()

In [35]:
omni.replicator.core.AnnotatorRegistry?

Init signature: omni.replicator.core.AnnotatorRegistry()
Docstring:      Registry of annotators providing groundtruth data to writers.
File:           ~/lab/robotodo/.conda/lib/python3.11/site-packages/omni/data/Kit/Isaac-Sim Full/5.0/exts/3/omni.replicator.core-1.12.17+107.3.1.lx64.r.cp311/omni/replicator/core/scripts/annotators.py
Type:           type
Subclasses:     

In [15]:
# TODO
import omni
omni.replicator.core.orchestrator.preview()

In [14]:
omni.replicator.core.orchestrator.get_status()

<Status.STOPPED: 0>

In [ ]:
omni.replicator.core.orchestrator.set_capture_on_play

In [38]:
omni.replicator.core.orchestrator.run()

In [16]:
kernel.get_settings().get("/app/player/playSimulations")


True

In [8]:
import omni

# TODO 
kernel.get_settings().set("/app/player/playSimulations", False)
timeline = omni.timeline.acquire_timeline_interface()
timeline.play()
timeline.stop()
kernel.get_settings().set("/app/player/playSimulations", True)


2025-10-11T11:13:03Z [30,964ms] [Error] [omni.physx.plugin] PhysicsUSD: Parse collision - triangle mesh collision (approximation None/MeshSimplification) cannot be a part of a dynamic body, falling back to convexHull approximation: /_02.For dynamic collision please use approximations : convex hull, convex decomposition, box, sphere or SDF approximation.
